In [ ]:
def CBHG(arg):
    tf.reset_default_graph()
    is_training = arg.is_training
    lr = arg.lr
    num_highway = arg.num_highway

    x = tf.placeholder(tf.float32,shape=(None,None, 256))
    y = tf.placeholder(tf.float32,shape=(None,None, 80))
    print(self.x.shape)
    #inputs shape:[Batch,T,256]

    inputs = tf.layers.dense(x,units=128,activation=None,use_bias=False,name='trans')

    convbanks_outputs = conv1d_banks(inputs,K=16,num_units=128,is_training=self.is_training)
    #output shape:[Batch,T,128*16]

    maxpool_outputs = tf.layers.max_pooling1d(convbanks_outputs,pool_size = 2,strides = 1,padding='same')
    #output shape:[Batch,T,128*16]

    c1 = Conv1d(inputs = maxpool_outputs,filters = 128,size = 3,activation=tf.nn.relu, scope='conv1d_1')
    #output shape:[Batch,T,128]


    n1 = tf.layers.batch_normalization(c1,training = self.is_training)
    #output shape:[Batch,T,128]

    c2 = Conv1d(inputs = n1, filters = 128, size = 3, activation=None, scope='conv1d_2')
    #output shape:[Batch,T,128]


    n2 = tf.layers.batch_normalization(c2,training = self.is_training)
    #output shape:[Batch,T,128]

    highway_inputs  = n2 + inputs
    #output shape:[Batch,T,128]

    for i in range(self.num_highway):
        highway_inputs = highway(highway_inputs,num_units=128,scope='highwaynet_{}'.format(i))
    #output shape:[Batch,T,128]

    bigru_outputs = gru_bi(inputs = highway_inputs,num_units = 128,scope='gru1')
    #output shape:[Batch,T,128]

    outputs = tf.layers.dense(bigru_outputs,units=80,activation=None,use_bias=False,name='O')
    #output shape:[Batch,T,80]
    
    return outputs
        
            #self.loss = tf.square(tf.subtract(outputs, self.y, name=None), name=None)
#             print(self.loss.shape)
            #c = tf.square(self.y - outputs)
            self.istarget = tf.to_float(tf.not_equal(self.y,tf.zeros_like(self.y)))
            #p = tf.reduce_sum(c * self.istarget,2)
            #s = tf.reduce_sum(p,1)
            #self.mean_loss = tf.reduce_mean(s)
            self.mean_loss = tf.losses.mean_squared_error(self.y,outputs*self.istarget)
#            print(self.mean_loss.shape)
           # self.istarget = tf.to_float(tf.not_equal(self.y, tf.zeros_like(self.y))) # masking
#             print(self.istarget)
            
           # self.mean_loss = tf.reduce_sum(self.loss * self.istarget) / 64.0
#             print(self.mean_loss.shape)
           

            self.global_step = tf.Variable(0,name='global_step',trainable=False)
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
            self.train_op = self.optimizer.minimize(self.mean_loss, global_step=self.global_step)
            
            tf.summary.scalar('loss',self.mean_loss)
            self.merged = tf.summary.merge_all()

In [ ]:
        if self.is_training:
            #self.loss = tf.square(tf.subtract(outputs, self.y, name=None), name=None)
#             print(self.loss.shape)
            #c = tf.square(self.y - outputs)
            self.istarget = tf.to_float(tf.not_equal(self.y,tf.zeros_like(self.y)))
            #p = tf.reduce_sum(c * self.istarget,2)
            #s = tf.reduce_sum(p,1)
            #self.mean_loss = tf.reduce_mean(s)
            self.mean_loss = tf.losses.mean_squared_error(self.y,outputs*self.istarget)
#            print(self.mean_loss.shape)
           # self.istarget = tf.to_float(tf.not_equal(self.y, tf.zeros_like(self.y))) # masking
#             print(self.istarget)
            
           # self.mean_loss = tf.reduce_sum(self.loss * self.istarget) / 64.0
#             print(self.mean_loss.shape)
           

            self.global_step = tf.Variable(0,name='global_step',trainable=False)
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
            self.train_op = self.optimizer.minimize(self.mean_loss, global_step=self.global_step)
            
            tf.summary.scalar('loss',self.mean_loss)
            self.merged = tf.summary.merge_all()